# Model Artifact Memory Consumption Tests

In [ ]:
SCHEDULER_IP=!kubectl get svc seldon-scheduler -n seldon-mesh -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
SCHEDULER_IP=SCHEDULER_IP[0]
import os
os.environ['SCHEDULER_IP'] = SCHEDULER_IP

In [ ]:
MESH_IP=!kubectl get svc seldon-mesh -n seldon-mesh -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
MESH_IP=MESH_IP[0]
import os
os.environ['MESH_IP'] = MESH_IP

## MLServer sklearn Iris

In [ ]:
numModels = 5
memory=!kubectl exec seldon-mlserver-0 -n seldon-mesh -c mlserver -- cat /sys/fs/cgroup/memory/memory.usage_in_bytes
memoryInitial=int(memory[0])
print(memoryInitial)

In [ ]:
for x in range(1,numModels+1):
    %env MODEL_NUM=$x
    !grpcurl -d '{"model":{ \
              "meta":{"name":"iris'$MODEL_NUM'"},\
              "modelSpec":{"uri":"gs://seldon-models/mlserver/iris",\
                           "requirements":["sklearn"],\
                           "memoryBytes":1},\
              "deploymentSpec":{"replicas":1}}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/LoadModel


In [ ]:
memory=!kubectl exec seldon-mlserver-0 -n seldon-mesh -c mlserver -- cat /sys/fs/cgroup/memory/memory.usage_in_bytes
memoryAfterLoad=int(memory[0])
print(memoryAfterLoad)

In [ ]:
for round in range(2):
    for x in range(1,numModels+1):
        %env MODEL_NUM=$x
        !curl -w "\n%{http_code}\n" http://${MESH_IP}/v2/models/iris$MODEL_NUM/infer -H "Content-Type: application/json" -H "Host: iris"$MODEL_NUM\
            -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

In [ ]:
memory=!kubectl exec seldon-mlserver-0 -n seldon-mesh -c mlserver -- cat /sys/fs/cgroup/memory/memory.usage_in_bytes
memoryAfterCurl=int(memory[0])
print(memoryAfterCurl)

In [ ]:
for x in range(1,numModels+1):
    %env MODEL_NUM=$x
    !grpcurl -d '{"model":{"name":"iris'$MODEL_NUM'"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/UnloadModel


In [ ]:
memory=!kubectl exec seldon-mlserver-0 -n seldon-mesh -c mlserver -- cat /sys/fs/cgroup/memory/memory.usage_in_bytes
memoryAfterUnload=int(memory[0])
print(memoryAfterUnload)

In [ ]:
memoryPerModel = ((memoryAfterCurl - memoryAfterUnload)/numModels) 
print(memoryPerModel)

## MLServer Xgboost Agaricus

In [ ]:
numModels = 5
memory=!kubectl exec seldon-mlserver-0 -n seldon-mesh -c mlserver -- cat /sys/fs/cgroup/memory/memory.usage_in_bytes
memoryInitial=int(memory[0])
print(memoryInitial)

In [ ]:
for x in range(1,numModels+1):
    %env MODEL_NUM=$x
    !grpcurl -d '{"model":{ \
              "meta":{"name":"agaricus'$MODEL_NUM'"},\
              "modelSpec":{"uri":"gs://seldon-models/mlserver/mushroom",\
                           "requirements":["sklearn"],\
                           "memoryBytes":1},\
              "deploymentSpec":{"replicas":1}}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/LoadModel


In [ ]:
memory=!kubectl exec seldon-mlserver-0 -n seldon-mesh -c mlserver -- cat /sys/fs/cgroup/memory/memory.usage_in_bytes
memoryAfterLoad=int(memory[0])
print(memoryAfterLoad)

In [ ]:
!grpcurl -d '{"model":{"name":"agaricus5"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ModelStatus

In [ ]:
for round in range(1):
    for x in range(1,numModels+1):
        print("")
        %env MODEL_NUM=$x
        !curl -w "\n%{http_code}\n" http://${MESH_IP}/v2/models/agaricus$MODEL_NUM/infer -H "Content-Type: application/json" -H "Host: agaricus"$MODEL_NUM\
            -d '{"inputs": [{"name": "predict","shape": [1, 126],"datatype": "FP32","data":[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]}]}'

In [ ]:
memory=!kubectl exec seldon-mlserver-0 -n seldon-mesh -c mlserver -- cat /sys/fs/cgroup/memory/memory.usage_in_bytes
memoryAfterCurl=int(memory[0])
print(memoryAfterCurl)

In [ ]:
for x in range(1,numModels+1):
    %env MODEL_NUM=$x
    !grpcurl -d '{"model":{"name":"agaricus'$MODEL_NUM'"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/UnloadModel


In [ ]:
memory=!kubectl exec seldon-mlserver-0 -n seldon-mesh -c mlserver -- cat /sys/fs/cgroup/memory/memory.usage_in_bytes
memoryAfterUnload=int(memory[0])
print(memoryAfterUnload)

In [ ]:
memoryPerModel = ((memoryAfterCurl - memoryAfterUnload)/numModels) 
print(memoryPerModel)

## MLServer Conclusion

 * Around 80M is used per Model - both models tested were smallish 1K (disk size) models.

## Triton Simple Tensorflow Model

In [ ]:
numModels = 20
memory=!kubectl exec seldon-triton-0 -n seldon-mesh -c triton -- cat /sys/fs/cgroup/memory/memory.usage_in_bytes
memoryInitial=int(memory[0])
print(memoryInitial)

In [ ]:
for x in range(1,numModels+1):
    %env MODEL_NUM=$x
    !grpcurl -d '{"model":{ \
              "meta":{"name":"simple'$MODEL_NUM'"},\
              "modelSpec":{"uri":"gs://seldon-models/triton/simple",\
                           "requirements":["tensorflow"],\
                           "memoryBytes":1},\
              "deploymentSpec":{"replicas":1}}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/LoadModel


In [ ]:
for x in range(1,numModels+1):
        %env MODEL_NUM=$x
        !grpcurl -d '{"model":{"name":"simple'$MODEL_NUM'"}}' \
             -plaintext \
             -import-path ../../apis \
             -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ModelStatus


In [ ]:
memory=!kubectl exec seldon-mlserver-0 -n seldon-mesh -c mlserver -- cat /sys/fs/cgroup/memory/memory.usage_in_bytes
memoryAfterLoad=int(memory[0])
print(memoryAfterLoad)

In [ ]:
for round in range(1):
    for x in range(1,numModels+1):
        %env MODEL_NUM=$x
        !curl -w "\n%{http_code}\n" http://${MESH_IP}/v2/models/simple$MODEL_NUM/infer -H "Content-Type: application/json" -H "Host: simple"$MODEL_NUM\
            -d '{"inputs":[{"name":"INPUT0","data":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16],"datatype":"INT32","shape":[1,16]},{"name":"INPUT1","data":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16],"datatype":"INT32","shape":[1,16]}]}'

In [ ]:
memory=!kubectl exec seldon-mlserver-0 -n seldon-mesh -c mlserver -- cat /sys/fs/cgroup/memory/memory.usage_in_bytes
memoryAfterCurl=int(memory[0])
print(memoryAfterCurl)

In [ ]:
for x in range(1,numModels+1):
    %env MODEL_NUM=$x
    !grpcurl -d '{"model":{"name":"simple'$MODEL_NUM'"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/UnloadModel


In [ ]:
memory=!kubectl exec seldon-mlserver-0 -n seldon-mesh -c mlserver -- cat /sys/fs/cgroup/memory/memory.usage_in_bytes
memoryAfterUnload=int(memory[0])
print(memoryAfterUnload)

In [ ]:
memoryPerModel = ((memoryAfterCurl - memoryAfterUnload)/numModels) 
print(memoryPerModel)

## Triton XGBoost Agaricus Model

In [ ]:
numModels = 20
memory=!kubectl exec seldon-triton-0 -n seldon-mesh -c triton -- cat /sys/fs/cgroup/memory/memory.usage_in_bytes
memoryInitial=int(memory[0])
print(memoryInitial)

In [ ]:
for x in range(1,numModels+1):
    %env MODEL_NUM=$x
    !grpcurl -d '{"model":{ \
              "meta":{"name":"triton-agaricus'$MODEL_NUM'"},\
              "modelSpec":{"uri":"gs://seldon-models/triton/agaricus",\
                           "requirements":["tensorflow"],\
                           "memoryBytes":1},\
              "deploymentSpec":{"replicas":1}}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/LoadModel


In [ ]:
for x in range(1,numModels+1):
        %env MODEL_NUM=$x
        !grpcurl -d '{"model":{"name":"triton-agaricus'$MODEL_NUM'"}}' \
             -plaintext \
             -import-path ../../apis \
             -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ModelStatus


In [ ]:
memory=!kubectl exec seldon-mlserver-0 -n seldon-mesh -c mlserver -- cat /sys/fs/cgroup/memory/memory.usage_in_bytes
memoryAfterLoad=int(memory[0])
print(memoryAfterLoad)

In [ ]:
for round in range(1):
    for x in range(1,numModels+1):
        %env MODEL_NUM=$x
        !curl -w "\n%{http_code}\n" http://${MESH_IP}/v2/models/triton-agaricus$MODEL_NUM/infer -H "Content-Type: application/json" -H "Host: triton-agaricus"$MODEL_NUM\
            -d '{"inputs": [{"name": "input__0","shape": [1, 126],"datatype": "FP32","data":[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]}]}'

In [ ]:
memory=!kubectl exec seldon-mlserver-0 -n seldon-mesh -c mlserver -- cat /sys/fs/cgroup/memory/memory.usage_in_bytes
memoryAfterCurl=int(memory[0])
print(memoryAfterCurl)

In [ ]:
for x in range(1,numModels+1):
    %env MODEL_NUM=$x
    !grpcurl -d '{"model":{"name":"triton-agaricus'$MODEL_NUM'"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/UnloadModel


In [ ]:
memory=!kubectl exec seldon-mlserver-0 -n seldon-mesh -c mlserver -- cat /sys/fs/cgroup/memory/memory.usage_in_bytes
memoryAfterUnload=int(memory[0])
print(memoryAfterUnload)

In [ ]:
memoryPerModel = ((memoryAfterCurl - memoryAfterUnload)/numModels) 
print(memoryPerModel)

## Triton Conclusions

 * Triton models take up variable small amount 0-10K for a small model of around 300 bytes on disk
   * Sometimes the memory used is negative by above calulation
   * Maybe due to delayed memory garbage collection?